In [247]:
!pip install text-hammer

In [248]:
!pip install imblearn

In [249]:
!pip install --upgrade scikit-learn

In [2]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer

# import imblearn
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler

from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [3]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [4]:
df = pd.read_csv('mbti_1.csv')
#df1=df[]
print(len(df))
df.head()

8675


,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [5]:
df=df.dropna()
df.reset_index(inplace=True)
df.head()
print(df.type)

0       INFJ
1       ENTP
2       INTP
3       INTJ
4       ENTJ
        ... 
8670    ISFP
8671    ENFP
8672    INTP
8673    INFP
8674    INFP
Name: type, Length: 8675, dtype: object


In [6]:
import text_hammer as th
import spacy
nlp = spacy.load("en_core_web_sm")

def text_preprocessing(df,col_name):
    column = col_name
    type(df[column])
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
    print(type(df[column]))
    print(1)
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x)) #you're -> you are; i'm -> i am
    print(type(df[column]))
    print(2)
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    print(type(df[column]))
    print(3)
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
    print(type(df[column]))
    print(4)
    df[column] = df[column].progress_apply(lambda x: th.remove_urls(x))
    print(df[column])
    print(4)
    df[column] = df[column].progress_apply(lambda x: th.remove_stopwords(x))
    print(type(df[column]))
    print(5)
    #df[column] = df[column].progress_apply(lambda x:th.spelling_correction(x))
    #print(type(df[column]))
    print(6)
    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(str(x)))
    print(type(df[column]))
    df[column]=pd.Series(df[column])
    print(7)
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
    print(type(df[column]))
    print(8)
    df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
    print(type(df[column]))
    print(9)
    return(df)

In [ ]:
# df=text_preprocessing(df1,"posts")

import time
# Record the start time
start_time = time.time()

# Your program code goes here
df=text_preprocessing(df,"posts")
# Record the end time
end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time

print(f"Time taken: {time_taken:.5f} seconds")

  0%|          | 0/8675 [00:00<?, ?it/s]

<class 'pandas.core.series.Series'>
1


  0%|          | 0/8675 [00:00<?, ?it/s]

In [ ]:
type(df)
print(df.iloc[:,1])
print(df.head(10))

In [ ]:
y = df.iloc[:,1].values
print("before: \n",y)
le=LabelEncoder()
y=le.fit_transform(y)
y=to_categorical(y)
for i in y[5]:
    if(i==1):
        print(list(y[5]).index(i))
        print(len(y[5]))
print("After: \n",y)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set()
# Convert the column to categorical type
df.iloc[:, 1] = pd.Categorical(df.iloc[:, 1])
t=list(set(df.iloc[:,1]))

# Plot the count of each category
sns.countplot(x=df.iloc[:, 1])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('punkt')
words_list = []
for post in df1['posts']:
    words_list.extend(nltk.word_tokenize(post))
freq_dist = nltk.FreqDist(words_list)
freq_dist.most_common(20)
temp = pd.DataFrame(freq_dist.most_common(30),  columns=['word', 'count'])
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(x='word', y='count', 
            data=temp, ax=ax)
plt.title("Top words")
plt.xticks(rotation='vertical')

In [ ]:
!pip install wordcloud
from wordcloud import WordCloud
import wordcloud
# creation of wordcloud
wcloud_fig = WordCloud( stopwords=set(wordcloud.STOPWORDS),
                      colormap='viridis', width=300, height=200).generate_from_frequencies(freq_dist)

# plotting the wordcloud
plt.figure(figsize=(10,7), frameon=True)

plt.imshow(wcloud_fig, interpolation  = 'bilinear')
plt.show()

In [ ]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
df.head()

In [ ]:
plt.figure(figsize=(15,10))
# sns.swarmplot("type", "words_per_comment", data=df)
sns.swarmplot(x="type", y="words_per_comment", data=df)

In [ ]:
voc_size = 50000
messages=df['posts'].copy()
messages

In [ ]:
ps=PorterStemmer()
corpus=[]
for i in range (0,len(messages)):
    review = re.sub('[^a-zA-Z]','',messages[i])
    review=review.lower()
    review=review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english') ]
    review=''.join(review)
    corpus.append(review)
#print(corpus)

In [ ]:
oe=[one_hot(words,voc_size) for words in corpus ]

In [ ]:
sent_length=250
embedded_docs = pad_sequences(oe,padding='pre',maxlen = sent_length)
embedded_docs

In [ ]:
df.type.value_counts()

# Model Training

In [ ]:
dimension=100
model = Sequential()
model.add(Embedding(voc_size,dimension,input_length = sent_length))
model.add(Dropout(0.25))
model.add(LSTM(100))
model.add(Dropout(0.25))
model.add(Dense(16,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['accuracy'])
model.summary()

In [ ]:
X=np.array(embedded_docs)

In [ ]:
ros = RandomOverSampler(random_state=42) # fit predictor and target variable
x_rus, y_rus = ros.fit_resample(X, y)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_rus, y_rus, test_size=0.2, random_state=42)
print((y_train[6]))

In [ ]:
y_train.shape
callbacks = [EarlyStopping( monitor="val_accuracy", patience=2 ), 
             ModelCheckpoint('model0.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)]

In [ ]:
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=5,batch_size=64, callbacks = callbacks )

In [ ]:
from sklearn.metrics import accuracy_score
print(len(x_test[0]))
preds = model.predict(x_test)
#print(preds)
#print(y_test)
o=np.argmax(preds,axis=1)
#for i in o:
#    print(t[i])
#print("Emotion:",len(y))

# Giving Input and Getting the Personality

In [1]:
s=input()
s =str(s).lower()
s=th.cont_exp(s) #you're -> you are; i'm -> i am
s=th.remove_emails(s)
s =th.remove_html_tags(s)
s=th.remove_urls(s)
s = th.remove_stopwords(s)
    #df[column] = df[column].progress_apply(lambda x:th.spelling_correction(x))
    #print(type(df[column]))
s =th.remove_special_chars(s)
s = th.remove_accented_chars(s)
s = th.make_base(s) #ran -> run,
print(s)
words_list = []
words_list.extend(nltk.word_tokenize(s))
freq_dist = nltk.FreqDist(words_list)
freq_dist.most_common(20)
temp = pd.DataFrame(freq_dist.most_common(30),  columns=['word', 'count'])
#fig, ax = plt.subplots(figsize=(10, 6))
#sns.barplot(x='word', y='count', data=temp, ax=ax)
ps=PorterStemmer()
corpus=[]
review = re.sub('[^a-zA-Z]','',s)
review=review.lower()
review=review.split()
review = [ps.stem(word) for word in review if not word in stopwords.words('english') ]
review=''.join(review)
corpus.append(review)
oe=[one_hot(words,voc_size) for words in corpus ]
sent_length=250
embedded_docs = pad_sequences(oe,padding='pre',maxlen = sent_length)
embedded_docs
X=np.array(embedded_docs)
p=model.predict(X)
o1=np.argmax(p,axis=1)
print("Personality:",t[o1[0]])

Hello


NameError: name 'th' is not defined

# Confusion Matrix

In [328]:
from sklearn.metrics import confusion_matrix
true_cat = []
for y in y_test:
     true_cat.append(np.where(y==1)[0])
predicted_cat = tf.argmax(preds, axis=1)
predicted_cat
print(confusion_matrix(predicted_cat, true_cat))

[[375   1   0   0   0   0   0   0   3   1   1   0   0   1   0   3]
 [  0 301   5   4   0   0   0   0   2   4   4   5   0   0   0   4]
 [  0   3 386   2   0   0   0   0   0   2   5   1   0   0   0   0]
 [  0   0   0 285   0   0   0   0  10   6   3   4   0   0   0   0]
 [  0   0   0   0 382   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 371   0   0   0   0   0   0   0   0   1   0]
 [  0   1   0   0   0   0 351   0   0   0   0   1   0   0   0   0]
 [  0   1   0   1   0   0   0 391   0   0   0   0   0   1   0   3]
 [  0   0   0   3   0   0   0   0 105   6   6   5   0   0   0   0]
 [  0  19   2  15   0   0   0   0  80 138  55  73   0   0   0   2]
 [  1  27   0  26   0   0   0   0 129 145 278  83   0   2   1   3]
 [  0   4   0   2   0   0   0   0  26  40  15 168   0   0   0   2]
 [  0   3   1   2   0   0   0   0   1   0   2   0 348   0   0   0]
 [  0   0   3   2   0   0   0   0   1   1   4   1   2 382   0   3]
 [  0   0   4   1   0   0   0   0   1   0   1   1   0   0 351 